In [1]:
# import os
# os.environ["NUMBA_DISABLE_JIT"] = "1"
# os.environ["NUMBA_DEBUGINFO"] = "1"
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import numpy as np
import pandas as pd
import ccxt

np.set_printoptions(precision=4)

# data = vbt.CCXTData.download(
#     "BTCUSDT",
#     start='2020-10-01 UTC',
#     end='2021-02-23 UTC',
#     timeframe='30m',
#     exchange='bybit',
# )
# data.save('test_long_cartesian.pickle')
print (ccxt.exchanges)

['ace', 'alpaca', 'ascendex', 'bequant', 'bigone', 'binance', 'binancecoinm', 'binanceus', 'binanceusdm', 'bit2c', 'bitbank', 'bitbay', 'bitbns', 'bitcoincom', 'bitfinex', 'bitfinex2', 'bitflyer', 'bitforex', 'bitget', 'bithumb', 'bitmart', 'bitmex', 'bitopro', 'bitpanda', 'bitrue', 'bitso', 'bitstamp', 'bitstamp1', 'bittrex', 'bitvavo', 'bkex', 'bl3p', 'blockchaincom', 'btcalpha', 'btcbox', 'btcex', 'btcmarkets', 'btctradeua', 'btcturk', 'buda', 'bybit', 'cex', 'coinbase', 'coinbaseprime', 'coinbasepro', 'coincheck', 'coinex', 'coinfalcon', 'coinmate', 'coinone', 'coinspot', 'cryptocom', 'currencycom', 'delta', 'deribit', 'digifinex', 'exmo', 'flowbtc', 'fmfwio', 'gate', 'gateio', 'gemini', 'hitbtc', 'hitbtc3', 'hollaex', 'huobi', 'huobijp', 'huobipro', 'idex', 'independentreserve', 'indodax', 'itbit', 'kraken', 'krakenfutures', 'kucoin', 'kucoinfutures', 'kuna', 'latoken', 'lbank', 'lbank2', 'luno', 'lykke', 'mercado', 'mexc', 'mexc3', 'ndax', 'novadax', 'oceanex', 'okcoin', 'okex', 

In [4]:
exchange = ccxt.bybit()  # default id
exchange.fetchOHLCV(
    symbol='BTCUSDT',
    timeframe='30m',
    params={
        'category': 'linear',
        'start': 1669852800000,
        'end': 1670852800000,  
    }
    )


[[1670493600000, 16792.0, 16820.0, 16792.0, 16812.0, 782.86],
 [1670495400000, 16812.0, 16821.0, 16804.5, 16813.5, 407.254],
 [1670497200000, 16813.5, 16818.5, 16800.0, 16817.0, 561.08],
 [1670499000000, 16817.0, 16850.5, 16702.5, 16840.5, 7026.49],
 [1670500800000, 16840.5, 16877.0, 16830.5, 16830.5, 3133.094],
 [1670502600000, 16830.5, 16853.5, 16817.0, 16818.0, 2016.842],
 [1670504400000, 16818.0, 16829.5, 16792.0, 16825.0, 1730.768],
 [1670506200000, 16825.0, 16860.0, 16807.0, 16850.0, 2306.485],
 [1670508000000, 16850.0, 16868.0, 16836.0, 16848.5, 1364.563],
 [1670509800000, 16848.5, 16920.0, 16822.5, 16892.5, 4852.772],
 [1670511600000, 16892.5, 16940.5, 16877.0, 16915.5, 5692.496],
 [1670513400000, 16915.5, 16948.0, 16897.0, 16907.0, 2724.243],
 [1670515200000, 16907.0, 16932.0, 16902.0, 16925.5, 1609.2],
 [1670517000000, 16925.5, 16945.5, 16910.0, 16937.0, 1321.023],
 [1670518800000, 16937.0, 16964.5, 16927.0, 16936.5, 2756.025],
 [1670520600000, 16936.5, 16956.5, 16920.5, 1695

In [2]:
rsi = vbt.RSI.run(
    close,
    window=[15, 20],
    param_product=True
)
entries = rsi.rsi_below([20, 22]).shift(1).fillna(False)

In [3]:
account_state = AccountAndTradeState_QF(
    available_balance=100.,
    equity=100.,
)
order = OrderEverything_QF(
    lev_mode=np.array([LeverageMode_QF.LeastFreeCashUsed]),
    order_type=np.array([OrderType_QF.LongEntry]),
    size_type=np.array([SizeType_QF.RiskPercentOfAccount]),
    size_pct=np.array([1.]),
    max_equity_risk_pct=np.arange(5, 6.1, 1),
)
all_stops = AllStops_QF(
    sl_pcts=np.arange(5, 6.1, .25),
    risk_rewards = np.arange(5, 6.1, .5),
)

In [4]:
df_array, order_records, _ = cart_from_signals(
    open=open,
    high=high,
    low=low,
    close=close,
    entries=entries,
    order=order,
    account_state=account_state,
    all_stops=all_stops,
)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1m[1m[1m[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1m[1m[1m[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<class 'numba.core.types.functions.NamedTupleClass'>) found for signature:
 
 >>> NamedTupleClass(lev_mode=float64, order_type=float64, price=float64, size_type=float64, allow_partial=float64, available_balance=float64, average_entry=float64, cash_borrowed=float64, cash_used=float64, equity=float64, fee=float64, fees_paid=float64, leverage=float64, liq_price=float64, log=float64, max_equity_risk=float64, max_equity_risk_pct=float64, max_lev=float64, max_order_pct_size=float64, max_order_size=float64, min_order_pct_size=floa

In [ ]:
df = pd.DataFrame(df_array)
pd.options.display.float_format = '{:,.2f}'.format

for idx, name in enumerate(entries.columns.names):
    df[name] = df['ind_set']
    for jjj in range(df['ind_set'].min(), df['ind_set'].max()+1):
        df[name].mask(df['ind_set'] == jjj, entries.columns[jjj][idx], inplace=True)
df.sort_values(by=['to_the_upside'], ascending=False).head(30)

In [ ]:
# print(df.sort_values(by=['to_the_upside'], ascending=False).head(10).to_markdown())